<a href="https://colab.research.google.com/github/saijahnavi9/LLM-Hands-on/blob/main/Jahnavi_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy faiss-cpu sentence-transformers streamlit openai langchain chromadb


In [2]:
import zipfile

# Path to the zip file
zip_path = "/archive (7).zip"

# Extract to a specific folder (optional)
extract_path = "/content/abc"

# Unzip the file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [3]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/abc/depression_dataset_reddit_cleaned.csv")

# Display the first few rows
print(df.head())


                                          clean_text  is_depression
0  we understand that most people who reply immed...              1
1  welcome to r depression s check in post a plac...              1
2  anyone else instead of sleeping more when depr...              1
3  i ve kind of stuffed around a lot in my life d...              1
4  sleep is my greatest and most comforting escap...              1


In [4]:
import re

# Function to preprocess text
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
        return text.strip()
    return ""

# Apply preprocessing to the clean_text column
df["Processed_Text"] = df["clean_text"].apply(preprocess_text)

# Display the first few rows of the processed dataset
print(df.head())


                                          clean_text  is_depression  \
0  we understand that most people who reply immed...              1   
1  welcome to r depression s check in post a plac...              1   
2  anyone else instead of sleeping more when depr...              1   
3  i ve kind of stuffed around a lot in my life d...              1   
4  sleep is my greatest and most comforting escap...              1   

                                      Processed_Text  
0  we understand that most people who reply immed...  
1  welcome to r depression s check in post a plac...  
2  anyone else instead of sleeping more when depr...  
3  i ve kind of stuffed around a lot in my life d...  
4  sleep is my greatest and most comforting escap...  


In [5]:
from sentence_transformers import SentenceTransformer

# Load pre-trained Sentence Transformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

print("Model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded successfully!


In [6]:
import numpy as np
from tqdm import tqdm

# Function to encode in batches
def batch_encode(texts, batch_size=1000):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i : i + batch_size]
        batch_embeddings = model.encode(batch, convert_to_numpy=True)  # Ensure model is already loaded
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

# Encode clean_text in batches
embeddings = batch_encode(df["clean_text"].tolist(), batch_size=500)

# Save embeddings for reuse
np.save("clean_text_embeddings.npy", embeddings)


100%|██████████| 16/16 [00:07<00:00,  2.06it/s]


In [7]:
import faiss

# Define vector dimension (should match model output)
embedding_dim = embeddings.shape[1]

# Create a FAISS index
index = faiss.IndexFlatL2(embedding_dim)  # L2 distance (Euclidean)
index.add(embeddings)  # Add embeddings to the index

print(f"FAISS index created with {index.ntotal} entries!")


FAISS index created with 7731 entries!


In [8]:
def retrieve_similar_reviews(query, top_k=5):
    # Convert the query to an embedding
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Search the FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Fetch corresponding rows from the dataset
    results = df.iloc[indices[0]]

    return results[["clean_text", "is_depression"]]  # Adjusted columns

# Example query test
query = "feeling sad and lonely"
results = retrieve_similar_reviews(query)

print("Top Matches:")
print(results)

Top Matches:
                                             clean_text  is_depression
5729                                i feel lonely today              0
5047                                   bored and lonely              0
5935                   feeling ill and sorry for myself              0
207    i need friend i am so sad and lonely please help              1
433   i never really experience sadness when depress...              1


In [9]:
# Example queries relevant to the dataset
print(retrieve_similar_reviews("Feeling extremely sad and hopeless"))
print(retrieve_similar_reviews("Looking for advice to overcome stress"))


                                             clean_text  is_depression
100   i think i m happy i ve just went through a cou...              1
506   i m failing out of college right now because i...              1
433   i never really experience sadness when depress...              1
3636  i ve always joked about depression but lately ...              1
157   i literally feel empty and hollow i feel like ...              1
                                             clean_text  is_depression
4629     my stress always culminates with physical pain              0
1801  there are a million thing to stress about righ...              1
6436  stuck on my own at work super stressed i wish ...              0
1212  i have an interview tomorrow and i feel like i...              1
1864  stress i love it like it bliss stress is so fu...              1


In [10]:
!pip install streamlit


In [11]:
%%writefile app.py
import torch
import streamlit as st
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pandas as pd

# Load the pre-trained SentenceTransformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset and FAISS index
df = pd.read_csv("/content/abc/depression_dataset_reddit_cleaned.csv")  # Replace with your file path
embeddings = np.load("clean_text_embeddings.npy")  # Pre-generated embeddings
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Function to retrieve similar posts
def retrieve_similar_posts(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    results = df.iloc[indices[0]]
    return results[["clean_text", "is_depression"]]

# Streamlit Interface
st.title("Depression Dataset Query Search")

# User Query Input
query = st.text_input("Enter your query about mental health:", "")
if query:
    # Display the search button
    if st.button("Search"):
        st.write("Searching for related posts...")

        # Perform search and retrieve results
        results = retrieve_similar_posts(query, top_k=5)

        # Display the results
        if not results.empty:
            st.subheader("Top Matches:")
            for idx, row in results.iterrows():
                st.write(f"**Post:** {row['clean_text']}")
                st.write(f"**Is Depression (Label):** {'Yes' if row['is_depression'] == 1 else 'No'}")
                st.write("---")
        else:
            st.write("No results found.")

# Optional Interactive Filters
st.sidebar.header("Filter Options")

# Checkbox for filtering by depression label
filter_depression = st.sidebar.checkbox("Only show posts labeled as depression", value=False)

# Apply filtering based on the checkbox
if filter_depression:
    df = df[df["is_depression"] == 1]

# Display additional exploration button
if st.button("Explore Related Content"):
    related_content = retrieve_similar_posts(query, top_k=10)
    st.write("Exploring related posts...")
    for idx, row in related_content.iterrows():
        st.write(f"**Post:** {row['clean_text']}")
        st.write(f"**Is Depression (Label):** {'Yes' if row['is_depression'] == 1 else 'No'}")
        st.write("---")

# Sidebar note for users
st.sidebar.write("Adjust search filters and explore posts based on your preferences.")


Writing app.py


In [7]:
!pip install streamlit pyngrok --quiet

In [8]:
!wget -q -O ngrok.zip https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [9]:
!streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.63.223:8501

  Stopping...
  Stopping...


In [10]:
!pip install pyngrok


In [11]:
!ngrok authtoken 2suwYfa5DzekeD6MInLkn2PJK0z_JxJndYjqY7XVwSiuQLLo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
!curl https://loca.lt/mytunnelpassword

34.125.63.223

In [13]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://smooth-terms-laugh.loca.lt
